In [1]:
import time
import json
import h5py
import numpy as np

In [2]:
def calc_value(n_pos, nation_vector, league_vector, club_vector, lc_vector, connection_n_vector, connection_matrix):
    value_matrix = np.zeros((n_pos, n_pos), dtype=int)
    for i in range(n_pos):
        if nation_vector[i] != 0:
            value_matrix[i, :] += np.equal(nation_vector[i], nation_vector)
        if league_vector[i] != 0:
            value_matrix[i, :] += np.equal(league_vector[i], league_vector)
            if club_vector[i] != 0:
                value_matrix[i, :] += np.equal(lc_vector[i], lc_vector)
    
    link_matrix = np.multiply(connection_matrix, value_matrix)
    link_sum_vector = link_matrix.sum(axis = 0)
    link_value_vector = np.divide(link_sum_vector,connection_n_vector)
    
    return link_value_vector

In [3]:
def NLC_interations(n_pos, iterations, connection_matrix, connection_n_vector, divisor_vector, product_vector):
    iteration_matrix = np.zeros((iterations, n_pos))
    
    for i in range(iterations):
        vectors = np.mod(np.full(3 * n_pos, i) // product_vector, divisor_vector)
        club_vector = vectors[0 : n_pos]
        league_vector = vectors[n_pos : 2 * n_pos]
        nation_vector = vectors[2 * n_pos : 3 * n_pos]
        lc_vector = club_vector + 100 * league_vector
        
        link_value_vector = calc_value(n_pos, nation_vector, league_vector, club_vector, lc_vector, 
                                       connection_n_vector, connection_matrix)
        iteration_matrix[i, :] = link_value_vector
    
    return iteration_matrix

In [4]:
f = open("../files/formations.json")
d = json.load(f)

In [5]:
n_N = 2
n_L = 2
n_C = 1

formation_index = 0

In [6]:
posisitons_list = d["formations"][formation_index]["positions_list"]
name = d["formations"][formation_index]["name"]
n_pos = len(posisitons_list)

In [7]:
iterations = (n_N * n_L * n_C)**n_pos

In [8]:
print(name)
print(iterations)

4-4-2
4194304


In [9]:
connection_matrix = np.zeros((n_pos,n_pos), dtype=int)
for i, pos in enumerate(posisitons_list):
    connection_matrix[i, pos["connections_list"]] = 1

In [10]:
connection_n_vector = connection_matrix.sum(axis = 0)

In [11]:
divisor_vector = np.full(n_pos, n_C)
divisor_vector = np.append(divisor_vector, np.full(n_pos, n_L))
divisor_vector = np.append(divisor_vector, np.full(n_pos, n_N))

In [12]:
product_vector = np.zeros(3 * n_pos)

prod = 1
for i in range(n_pos):
    product_vector[i] = prod
    prod *= n_C

for i in range(n_pos):
    product_vector[i + n_pos] = prod
    prod *= n_L
    
for i in range(n_pos):
    product_vector[i + 2 * n_pos] = prod
    prod *= n_N

In [13]:
start_time = time.time()
iteration_matrix = NLC_interations(n_pos, iterations, connection_matrix, connection_n_vector, divisor_vector, product_vector)
print("--- %s seconds ---" % (time.time() - start_time))

--- 286.86546087265015 seconds ---


In [15]:
filename = "C:/Users/Truls/OneDrive/Documents/02 - Privat/Hobby/Python/FUT/hybrid_formations/" + name + ", N{}L{}C{}.h5".format(n_N, n_L, n_C)
h5f = h5py.File(filename, 'w')
h5f.create_dataset('dataset_1', data=iteration_matrix)
h5f.close()